### imports

In [1]:
import sys
import os
import time
from tables import *
from datetime import datetime
import importlib


# Get the current working directory - assuming athina working directory here
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")
### change the current working directory if necessary
#os.chdir('..')
#current_directory = os.getcwd()
#print(f"Current working directory: {current_directory}")

# Add the parent directory 'HogProf/src/HogProf' to sys.path (for importing lshbuilder)
sys.path.append(os.path.abspath('HogProf/src/HogProf'))

# Import lshbuilder module from the parent directory
try:
    import lshbuilder
    importlib.reload(lshbuilder)
    print("Successfully imported lshbuilder.")
except ImportError as e:
    print(f"Error importing lshbuilder: {e}")
# Import profiler module from the parent directory
try:
    import profiler
    importlib.reload(profiler)
    print("Successfully imported profiler.")
except ImportError as e:
    print(f"Error importing profiler: {e}")

# Add the utils directory 'HogProf/src/HogProf/utils' to sys.path (for importing phyhamutils and hashutils)
sys.path.append(os.path.abspath('HogProf/src/HogProf/utils'))

# Import phyhamutils and hashutils from the utils directory
try:
    import pyhamutils
    importlib.reload(pyhamutils)
    import hashutils
    importlib.reload(hashutils)
    print("Successfully imported pyhamutils and hashutils.")
except ImportError as e:
    print(f"Error importing pyhamutils or hashutils: {e}")


Current working directory: /work/FAC/FBM/DBC/cdessim2/default/agavriilidou
Successfully imported lshbuilder.
Successfully imported profiler.
Successfully imported pyhamutils and hashutils.


### lshbuilder main function

In [2]:
def lshbuilder_main(dbname, orthoglob=None, nperm=256, omafile=None, taxfilter=None, taxmask=None, weights=None, 
mastertree=None, lossonly=False, duplonly=False, taxcodes=False, verbose=False, reformat_names=False, threads=4,
dbtype=None):
    '''
    parser = argparse.ArgumentParser()
    parser.add_argument('--taxweights', help='load optimised weights from keras model',type = str)
    parser.add_argument('--taxmask', help='consider only one branch',type = str)
    parser.add_argument('--taxfilter', help='remove these taxa' , type = str)
    parser.add_argument('--outpath', help='name of the db', type = str)
    parser.add_argument('--dbtype', help='preconfigured taxonomic ranges' , type = str)
    parser.add_argument('--OMA', help='use oma data ' , type = str)
    parser.add_argument('--OrthoGlob', help='a glob expression for orthoxml files ' , type = str)
    parser.add_argument('--tarfile', help='use tarfile with orthoxml data ' , type = str)
    parser.add_argument('--nperm', help='number of hash functions to use when constructing profiles' , type = int)
    parser.add_argument('--mastertree', help='master taxonomic tree. nodes should correspond to orthoxml' , type = str)
    
    parser.add_argument('--nthreads', help='nthreads for multiprocessing' , type = int)
    parser.add_argument('--lossonly', help='only compile loss events' , type = bool)
    parser.add_argument('--duplonly', help='only compile duplication events' , type = bool)
    parser.add_argument('--taxcodes', help='use taxid info in HOGs' , type = str)
    parser.add_argument('--verbose', help='print verbose output' , type = bool)
    parser.add_argument('--reformat_names', help='try to correct broken species trees by replacing all names with numbers.' , type = bool)
    '''
    ### the following dictionary may be outdated or does not work with reformat_names (14.10.2024)
    dbdict = {
        'all': { 'taxfilter': None , 'taxmask': None },
        'plants': { 'taxfilter': None , 'taxmask': 33090 },
        'archaea':{ 'taxfilter': None , 'taxmask': 2157 },
        'bacteria':{ 'taxfilter': None , 'taxmask': 2 },
        'eukarya':{ 'taxfilter': None , 'taxmask': 2759 },
        'protists':{ 'taxfilter': [2 , 2157 , 33090 , 4751, 33208] , 'taxmask':None },
        'fungi':{ 'taxfilter': None , 'taxmask': 4751 },
        'metazoa':{ 'taxfilter': None , 'taxmask': 33208 },
        'vertebrates':{ 'taxfilter': None , 'taxmask': 7742 },
    }

    if dbtype:
        taxfilter = dbdict[dbtype]['taxfilter']
        taxmask = dbdict[dbtype]['taxmask']
        print('using dbtype', dbtype, 'with taxfilter', taxfilter, 'and taxmask', taxmask)

    if weights:
        from keras.models import model_from_json
        json_file = open(  args['taxweights']+ '.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights(  args['taxweights']+".h5")
        print("Loaded model from disk")
        weights = model.get_weights()[0]
        weights += 10 ** -10

    start = time.time()
    if omafile:
        with open_file( omafile , mode="r") as h5_oma:
            lsh_builder = lshbuilder.LSHBuilder(h5_oma = h5_oma,  fileglob=orthoglob ,saving_name=dbname , numperm = nperm ,
            treeweights= weights , taxfilter = taxfilter, taxmask=taxmask , masterTree =mastertree , 
            lossonly = lossonly , duplonly = duplonly , use_taxcodes = taxcodes , reformat_names=reformat_names, verbose=verbose )
            #### maybe here is where load_one and saver are needed instaed of the run_pipeline!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            lsh_builder.run_pipeline(threads)
    else:
        lsh_builder = lshbuilder.LSHBuilder(h5_oma = None,  fileglob=orthoglob ,saving_name=dbname , numperm = nperm ,
        treeweights= weights , taxfilter = taxfilter, taxmask=taxmask ,
          masterTree =mastertree , lossonly = lossonly , duplonly = duplonly , use_taxcodes = taxcodes , reformat_names=reformat_names, verbose=verbose)
        lsh_builder.run_pipeline(threads)
    print(time.time() - start)
    print('DONE')

### testing

In [3]:
today = datetime.now().strftime("%y%m%d")
### 1 thread needs more than 2 ours for Toxicofera taxmask - do not let run from here
args = {
    'dbname': f'/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/venom_project/2a_hogprof_testing/test_{today}_',
    #'orthoglob': None,
    'omafile': '/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/oma_downloads/OmaServer.h5',
    #'taxweights': None,
    'taxmask': 'Toxicofera',            # 'Toxicofera', #4115 taxa after taxmask 
    #'taxfilter': None,
    #'dbtype': 'vertebrates',   ###does not seem to work, taxmask seems to be ignored later in the pipeline
    #'nperm': 256,
    'mastertree': '/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/oma_downloads/speciestree.nwk',
    'threads': 1,
    #'lossonly': False,
    #'duplonly': False,
    #'taxcodes': True,        ### does not seem to work due to bacteria and archaea not being in the taxcodes
    'verbose': True,
    'reformat_names': True    ### seems to be necessary or broken newick file errors appear
}
lshbuilder_main(**args)

initializing LSHBuilder
reformatted tree
taxmask Toxicofera
changed tree string (3844:1,(3904:1,3905:1)3845:1,(((4005:1,4006:1)3956:1,(4007:1,4008:1)3957:1,(4009:1,4010:1)3958:1)3906:1,3907:1)3846:1)3778:1;
Number of nodes before change: 4115
making tree weights w n taxa = : 17

configuring pyham functions
swap ids False
reformat names True
use phyloxml False
use taxcodes False
reading oma hdf5 with n groups: 1040435
done

run w n threads: 1
start workers


0it [00:00, ?it/s]

worker init 0creating dataset
generating dataframes

filtered at taxonomic level: NoFilter_Mask3778
{}
saver init 0


1it [00:00,  1.59it/s]

No top level hogs found for family 790033.
<orthoXML origin="OMA" originVersion="Jun 2024" version="0.5">
  <species name="4010" NCBITaxId="103944" taxonId="1352">
    <database name="Protobothrops mucrosquamatus from Refseq" version="Refseq; P.Mucros_1.0; GCF_001527695.2; 23-MAY-2019; Protobothrops mucrosquamatus Annotation Release 101">
      <genes>
        <gene id="15787594" protId="PROMU09585" />
      </genes>
    </database>
  </species>
  <species name="3904" NCBITaxId="28377" taxonId="1341">
    <database name="Anolis carolinensis from ENSEMBL v70" version="Ensembl 70; AnoCar2.0; 11-DEC-2012">
      <genes>
        <gene id="15537855" protId="ANOCA07078" />
      </genes>
    </database>
  </species>
  <species name="4007" NCBITaxId="8673" taxonId="1348">
    <database name="Pseudonaja textilis from Ensembl Vertebrates 51" version="Ensembl Vertebrates 51; EBS10Xv2-PRI">
      <genes>
        <gene id="15669188" protId="PSETE00760" />
      </genes>
    </database>
  </species

...

No top level hogs found for family 805253.
<orthoXML origin="OMA" originVersion="Jun 2024" version="0.5">
  <species name="4010" NCBITaxId="103944" taxonId="1352">
    <database name="Protobothrops mucrosquamatus from Refseq" version="Refseq; P.Mucros_1.0; GCF_001527695.2; 23-MAY-2019; Protobothrops mucrosquamatus Annotation Release 101">
      <genes>
        <gene id="15793172" protId="PROMU15163" />
      </genes>
    </database>
  </species>
  <species name="3904" NCBITaxId="28377" taxonId="1341">
    <database name="Anolis carolinensis from ENSEMBL v70" version="Ensembl 70; AnoCar2.0; 11-DEC-2012">
      <genes>
        <gene id="15531878" protId="ANOCA01101" />
      </genes>
    </database>
  </species>
  <species name="3907" NCBITaxId="176946" taxonId="1344">
    <database name="Python bivittatus from Refseq" version="Refseq; Python_molurus_bivittatus-5.0.2; GCF_000186305.1; 24-MAY-2018; Python bivittatus Annotation Release 102">
      <genes>
        <gene id="15803197" protId

5it [00:03,  1.43it/s]

No top level hogs found for family 761306.
<orthoXML origin="OMA" originVersion="Jun 2024" version="0.5">
  <species name="3907" NCBITaxId="176946" taxonId="1344">
    <database name="Python bivittatus from Refseq" version="Refseq; Python_molurus_bivittatus-5.0.2; GCF_000186305.1; 24-MAY-2018; Python bivittatus Annotation Release 102">
      <genes>
        <gene id="15802620" protId="PYTBI01260" />
      </genes>
    </database>
  </species>
  <species name="3904" NCBITaxId="28377" taxonId="1341">
    <database name="Anolis carolinensis from ENSEMBL v70" version="Ensembl 70; AnoCar2.0; 11-DEC-2012">
      <genes>
        <gene id="15535171" protId="ANOCA04394" />
      </genes>
    </database>
  </species>
  <species name="3844" NCBITaxId="61221" taxonId="1340">
    <database name="Varanus komodoensis from Ensembl 107" version="Ensembl 107; ASM479886v1; GCA_004798865.1">
      <genes>
        <gene id="15522519" protId="VARKO20923" />
      </genes>
    </database>
  </species>
  <spe

5it [00:04,  1.22it/s]Process Process-2:
Process Process-1:
Traceback (most recent call last):
  File "/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/miniconda3/envs/hogprof/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/miniconda3/envs/hogprof/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/HogProf/src/HogProf/lshbuilder.py", line 360, in saver
    this_dataframe = retq.get()
                     ^^^^^^^^^^
Traceback (most recent call last):
  File "/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/miniconda3/envs/hogprof/lib/python3.12/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
          ^^^^^^^^^^^^^^^^^^
  File "/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/miniconda3/envs/hogprof/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap

KeyboardInterrupt: 

### check results

In [4]:
hogprof_test_dir = '/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/venom_project/2a_hogprof_testing'
outfileprefix = '241017_toxicofera_newpyham_'
### get all output files
outfiles = [os.path.join(hogprof_test_dir, f) for f in os.listdir(hogprof_test_dir) if f.startswith(outfileprefix)]
### tidy up and print outfiles
print("Found output files:")
outfiles_dict = {}
filetypes = 'errors,hashes,idmapper,newlshforest,reformatted_tree,taxaIndex,wmg'.split(',')
for type in filetypes:
    for f in outfiles:
        if type in os.path.basename(f):
            outfiles_dict[type] = f
            print(type, os.path.basename(f))
            break
### check profiler object
print("\nLoading profiler object:")
p = profiler.Profiler(lshforestpath=outfiles_dict['newlshforest'], 
                      hashes_h5=outfiles_dict['hashes'], 
                      mat_path=os.path.join(hogprof_test_dir, outfileprefix),
                      oma='/work/FAC/FBM/DBC/cdessim2/default/agavriilidou/oma_downloads/OmaServer.h5',
                      nsamples=256,
                      mastertree=outfiles_dict['reformatted_tree'])

print("\nCheck the shape of the first dataset in the HDF5 file!")

### function to grab HOGs by protein ID
def grabHog(ID, verbose = True):
    try:
        entry = p.db_obj.entry_by_entry_nr(p.db_obj.id_resolver.resolve(ID))
        #print(entry)
        return entry
    except:
        return np.nan,np.nan


Found output files:
errors 241017_toxicofera_newpyham_errors.txt
hashes 241017_toxicofera_newpyham_hashes.h5
idmapper 241017_toxicofera_newpyham_idmapper.pkl
newlshforest 241017_toxicofera_newpyham_newlshforest.pkl
reformatted_tree 241017_toxicofera_newpyham_reformatted_tree.nwk
taxaIndex 241017_toxicofera_newpyham_taxaIndex.pkl
wmg 241017_toxicofera_newpyham_wmg.pkl

Loading profiler object:
loading lsh
indexing lsh
h5 <HDF5 file "241017_toxicofera_newpyham_hashes.h5" (mode r)> <KeysViewHDF5 ['NoFilter_Mask3778']>
first dataset in h5 file has shape (100, 0)
using newick
making tree weights w n taxa = : 2929
DONE

Check the shape of the first dataset in the HDF5 file!


In [5]:
### testing from PhylogeneticGraphAnalysis.ipynb
hog_id_toxicofera = [x[0] for x in p.db_obj.get_all_hogs_at_level('Toxicofera')]
print(f"Number of HOGs in Toxicofera: {len(hog_id_toxicofera)}")
print("Example HOG:", hog_id_toxicofera[0])
### generate hits for a HOG 
hits = p.hog_query(hog_id=hog_id_toxicofera[0], k=10)
print(f"Number of hits in the HOG: {len(hits)}")

Number of HOGs in Toxicofera: 31744
Example HOG: 706758
no dataset specified, using first dataset in the hdf5 file


IndexError: Index (1032346) out of range for (0-99)